In [ ]:
!pip install -q --upgrade crewai langchain langchain-community openai


In [ ]:
import nltk
nltk.download('punkt')


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"  # replace with your OpenAI key


In [ ]:
from crewai import Agent
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0.2)

web_scraper = Agent(
    role="Web Scraper",
    goal="Extract latest local news articles",
    backstory="You crawl news websites and return their titles and full text.",
    verbose=True,
    llm=llm
)

summarizer = Agent(
    role="Summarizer",
    goal="Summarize news articles into concise points",
    backstory="You convert long articles into short summaries for a newsletter.",
    verbose=True,
    llm=llm
)

classifier = Agent(
    role="Category Classifier",
    goal="Categorize each article",
    backstory="You classify news articles into categories like politics, sports, or tech.",
    verbose=True,
    llm=llm
)


In [ ]:
from crewai import Task

scrape_task = Task(
    description="Scrape the latest local news from The Hindu or NDTV and return title + article body.",
    expected_output="A list of news articles with their titles and full text.",
    agent=web_scraper
)

summarize_task = Task(
    description="Summarize each article retrieved by the Web Scraper.",
    expected_output="A list of summaries for each article.",
    agent=summarizer,
    depends_on=[scrape_task]
)

classify_task = Task(
    description="Categorize each summarized article into appropriate categories.",
    expected_output="A list of categories (e.g., Politics, Sports, Tech) corresponding to each summary.",
    agent=classifier,
    depends_on=[summarize_task]
)


In [ ]:
from crewai import Crew

crew = Crew(
    agents=[web_scraper, summarizer, classifier],
    tasks=[scrape_task, summarize_task, classify_task],
    verbose=True
)

result = crew.kickoff()
print(result)
